## Fonctions pour projet

need to normalize data ? (cf Dissecting financial markets: Sectors and states) (on devrait utiliser les returns donc pas trop besoin de normaliser)
- si on cluster les assets, logiquement la correlation/covariance entre chaque cluster devrait être négative/nulle (parce que la distance serait sans doute la covariance entre les assets), du coup faire une stratégie qui investit dans un représentant (asset ou weigted average) de chaque cluster devrait réduire le risque/volatilité du portfolio (problème étant que on aurait pas un zero-cost portfolio), ou alors prédire le signe de return de chaque cluster à la prochaine période et donc investir en fonction)
- si on cluster par jour/période de temps, l'idée serait de prédire le futur état et pour chaque état définir une méthode d'investissement
- https://d-nb.info/1108447864/34 papier qui parle de cluster les assets: truc important à noter est que les weights pour chaque cluster sont définis par hierarchical clustering p47-48, et les clusters/weights sont recalculés tous les 130 jours (permet d'éviter de refaire les clusters à chaque fois, et en pls ne suppose plus que les clusters restent les memes)

In [1]:

#Pour faire des clusters
#ce qui doit etre cluster sont les colonnes du dataframe (donc si on veut cluster les dates il faut transposer la matrice)


from numpy import linalg as LA
import numpy as np
import pandas as pd
import math
import networkx as nx
import community

#Idea is to remove low eigenvalues that are considered as noise
def compute_C_minus_C0(lambdas,v,lambda_plus,removeMarketMode=True): 
    N=len(lambdas)
    C_clean=np.zeros((N, N))
    
    order = np.argsort(lambdas)
    lambdas,v = lambdas[order],v[:,order]
    
    v_m=np.matrix(v)

    # note that the eivenvalues are sorted
    for i in range(1*removeMarketMode,N):                            
        if lambdas[i]>lambda_plus: 
            C_clean=C_clean+lambdas[i] * np.dot(v_m[:,i],v_m[:,i].T)  
    return C_clean    
    
    
def LouvainCorrelationClustering(R):   # R is a matrix of return
    N=R.shape[1]
    T=R.shape[0]

    q=N*1./T
    lambda_plus=(1.+np.sqrt(q))**2

    C=R.corr()
    lambdas, v = LA.eigh(C)


            
    C_s=compute_C_minus_C0(lambdas,v,lambda_plus)
    
    mygraph= nx.from_numpy_matrix(np.abs(C_s))
    partition = community.community_louvain.best_partition(mygraph,random_state=29)

    DF=pd.DataFrame.from_dict(partition,orient="index",columns=['Cluster'])
    DF.index=R.columns
    return(DF)





peut etre qu'on peut faire une analyse des eigenvalues et eigenvectors (cf cours 9-10)
mais la question est qu'on peut pas faire les eigenvalues sur tout le dataset (parce que dans ce cas on analyserait le futur avant meme de faire des clusters par exemple...)

De plus, cf cours 11 on pourrait faire varier les méthodes de cluster, que ce soit en changeant la facon de faire le cluster ou de changer le critère qui doit être cluster (return, sign of return, excess return ...)

et faudrait aussi du coup faire des investment strategies et les comparer entre elles, et avec un benchmark

In [2]:
#Creer un rolled clustering 
#(Suppose que l'index du dataframe est simplement des valeurs et non des dates)-> normalement mtn c bon pour tout
#Check si le cluster a pas déjà été fait et sauvegardé
#Si non il fait le cluster et le sauvegarde (économie de temps)

from os.path import exists
import os
from fastparquet import write, ParquetFile

#R is matrix of return
#cluster_period is the period over we cluster
#day_state is to know if we have to transpose matrix to cluster periods instead of assets
#lag in "if" is the number of time we cluster

def RolledCluster(R, cluster_period, filepath, day_state=False):
    liste=[]
    if day_state :
        for lag in range(1,20):  #on va devoir changer le range pour le projet mais la c'est pour tester
            t_0=R.index[0+lag]
            t_1=R.index[0+lag+cluster_period]    
            filename=filepath+"/louvain_cluster/{}_{}_cluster_day.parquet".format(t_0,t_1)
#Check if we already have the clustering or not
            if(exists(filename) and os.path.getsize(filename)>0):
                DF=pd.read_parquet(filename)
            else:
                rolling_data=R.iloc[(R.index>=t_0) & (R.index<t_1)]
                rolling_data=rolling_data.T
                DF= LouvainCorrelationClustering(rolling_data)
                DF.to_parquet(filename)
            liste.append(DF)
    else :
        for lag in range(1,20):  #on va devoir changer le range pour le projet mais la c'est pour tester
            t_0=R.index[0+lag]
            t_1=R.index[0+lag+cluster_period]    
            filename=filepath+"/louvain_cluster/{}_{}_cluster.parquet".format(t_0,t_1)
        
            if(exists(filename) and os.path.getsize(filename)>0):
                DF=pd.read_parquet(filename)
            else:
                rolling_data=R.iloc[(R.index>=t_0) & (R.index<t_1)]
                DF= LouvainCorrelationClustering(rolling_data)
                DF.to_parquet(filename)
            liste.append(DF)
    return(liste)

#Renvoie une liste de clusters
#Choose the assets (thus N ), choose the calibration length 
#(T = N/3 is a good choice for market states, T = 3N is a good choice for asset classification)

In [3]:
#Compute the rolling ARI of the clusters
from sklearn import metrics

def RolledARI(liste):
    ARI=[]
    for element in range(0, len(liste)-1):
        new_df=pd.merge(liste[element], liste[element+1], left_index=True, right_index=True)
        ARI.append(metrics.adjusted_rand_score(new_df["Cluster_x"], new_df["Cluster_y"]))
    return(ARI)

In [4]:
#fonction qu'on avait fait mais pas sur que ca serve dans notre cas
def compute_R(events,tau_max=1000,dtau=1):
    taus=range(1,tau_max,dtau)
    
    ## insert code here
    R=[-(events["mid"].diff(-tau)*events["s"]).mean() for tau in taus]
    
    
    return np.array(R)

def compute_R_h(events):
    R=pd.pivot_table(events.groupby(events.index.hour).apply(compute_R).apply(pd.Series), columns=events.groupby(events.index.hour).apply(compute_R).apply(pd.Series).index)
    return (R)

# Pour run les fonctions et les tester

In [5]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
logret= pd.read_parquet("/Users/mrobaux/Desktop/Financial big data/data/clean/equities/daily/us-stocks/us_logret.parquet")


new_logret=logret.loc[logret.index>10000]
new_logret= new_logret.loc[new_logret.index<16000]
dropcol = new_logret.loc[:,new_logret.isna().sum(axis=0)>0].columns
new_logret=new_logret.drop(columns=dropcol)

In [126]:
lii= RolledCluster(new_logret,90,"/Users/mrobaux/Desktop/Financial big data/projet",True)
a=RolledARI(lii)
number_of_clusters=[]
for element in lii:
    number_of_clusters.append(element["Cluster"].unique().max()+1)

In [65]:
#rename clusters based on similarity
#pour linstant renvoie simplement une matrice de similarite
#l'entree (i,j) correspond au pourcentage d'éléments du cluster 2j qui appartiennent aussi au cluster 1i
def keep_cluster_number(cluster1,cluster2):
    n=cluster1["Cluster"].unique().max()
    m=cluster2["Cluster"].unique().max()
    cst=10000 #pour éviter de changer deux fois les indices plus tard
    matrix = np.zeros((n+1,m+1))
    for i in range(n+1):
        for j in range(m+1):
            merged=pd.merge(cluster1,cluster2, left_index=True, right_index=True)
            num= len(merged[(merged["Cluster_y"]==j) & (merged["Cluster_x"]==i)])
            denom=len(merged[(merged["Cluster_y"]==j)])
            matrix[i][j]= num/denom
    tmp=cluster2.copy(deep=True) 
    for j in range(m+1): #Now change cluster number according to similarity
        value_cluster= np.where(matrix==np.amax(matrix,0))[1][j]
        tmp[tmp["Cluster"]==j]= (value_cluster+cst)
    tmp=tmp-cst
    return(tmp)
    

In [105]:
#One time prediction of cluster belonging
#lines = state t
#col = state t+1
def one_step_pred(cluster):
    m=cluster["Cluster"].unique().max()
    matrix = np.zeros((m+1,m+1))
    for p in range(len(cluster)-1):
        i=cluster["Cluster"].iloc[p]
        tmp=p+1
        j=cluster["Cluster"].iloc[tmp]
        matrix[i][j]=matrix[i][j]+1  #Counts number of times when you go from state i to state j
    matrix=matrix/(len(cluster)-1)
    return(matrix)

In [128]:
for i in range(len(lii)-1):
    lii[i+1]=keep_cluster_number(lii[i],lii[i+1])
    matrix = np.zeros((4,4))
for cluster in lii:
    matrix=matrix+one_step_pred(cluster)
matrix=matrix/len(lii)